# Connectome Analyses

We develop tools to analyze a circuit model's connectome.

In [2]:
import os
import numpy as np
import pandas as pd
from neuro_dmt.library.users.visood.sscx_dissemination.analyses.composition\
    import CompositionAnalysesSuite
from neuro_dmt.library.users.visood.sscx_dissemination.analyses.connectome\
    import ConnectomeAnalysesSuite
from neuro_dmt.models.bluebrain.circuit.atlas import\
    BlueBrainCircuitAtlas
from neuro_dmt.models.bluebrain.circuit.model import\
    BlueBrainCircuitModel,\
    CircuitProvenance
from neuro_dmt.models.bluebrain.circuit.adapter import\
    BlueBrainCircuitAdapter

In [3]:
project =\
    os.path.join(
        "/gpfs/bbp.cscs.ch/project")
proj_sscx_diss=\
    os.path.join(
        project,
        "proj83")
data_sscx_diss=\
    os.path.join(
        proj_sscx_diss,
        "data")
atlases_sscx_diss=\
    os.path.join(
        data_sscx_diss,
        "atlas/S1/MEAN")
circuits_sscx_diss=\
    os.path.join(
        proj_sscx_diss,
        "circuits")
atlas_bio_m =\
    BlueBrainCircuitAtlas(
        path=os.path.join(
                atlases_sscx_diss, "P14-MEAN"))
path_bio_m =\
    os.path.join(
        circuits_sscx_diss,
        "Bio_M/20191206")
circuit_bio_m =\
    BlueBrainCircuitModel(
        path_circuit_data=path_bio_m,
            provenance=CircuitProvenance(
                label="SSCxRatDisseminationBioM",
                authors=["BBP Team"],
                release_date="20191212",
                uri=path_bio_m,
                animal="Wistar Rat",
                age="P14",
                brain_region="SSCx"))
adapter =\
    BlueBrainCircuitAdapter()
suite_connectome_analyses =\
    ConnectomeAnalysesSuite(
        sample_size=20)

## Mock circuit

We will use a mock-circuit to test-develop connectome analyses.

In [4]:
from neuro_dmt.models.bluebrain.circuit.mock.circuit import\
    MockCircuit
from neuro_dmt.models.bluebrain.circuit.mock.test.mock_circuit_light import\
    circuit_composition,\
    circuit_connectivity

bluepy_mock_circuit =\
    MockCircuit.build(
        circuit_composition,
        circuit_connectivity)
mock_circuit_model =\
    BlueBrainCircuitModel(
        bluepy_mock_circuit,
        label="BlueBrainCircuitModelMockLight")

test develop mock circuit Logger                                                
DEBUGINFO@<2020-02-18 13:36:54>                                                 
Traceback:
	filename: 	/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/models/bluebrain/circuit/mock/test/__init__.py
	lineno: 	136
	code_context: 	['        __log.get_source_info(),\n']
	index: 0

get excitatory cell density for layer 2
with number of excitatory mtypes: 1

test develop mock circuit Logger                                                
DEBUGINFO@<2020-02-18 13:36:54>                                                 
Traceback:
	filename: 	/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/models/bluebrain/circuit/mock/test/__init__.py
	lineno: 	136
	code_context: 	['        __log.get_source_info(),\n']
	index: 0

get excitatory cell density for layer 3
with number of excitatory mtypes: 1

test develop mock circuit Logger                                    

test develop mock circuit Logger                                                
DEBUGINFO@<2020-02-18 13:36:54>                                                 
Traceback:
	filename: 	/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/models/bluebrain/circuit/mock/test/mock_circuit_light.py
	lineno: 	104
	code_context: 	['        log.get_source_info(),\n']
	index: 0

get excitatory cell density for layer 5
with number of excitatory mtypes: 3

test develop mock circuit Logger                                                
DEBUGINFO@<2020-02-18 13:36:54>                                                 
Traceback:
	filename: 	/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/models/bluebrain/circuit/mock/test/mock_circuit_light.py
	lineno: 	104
	code_context: 	['        log.get_source_info(),\n']
	index: 0

get excitatory cell density for layer 5
with number of excitatory mtypes: 3

test develop mock circuit Logger                    

test develop mock circuit Logger                                                
DEBUGINFO@<2020-02-18 13:36:54>                                                 
Traceback:
	filename: 	/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/models/bluebrain/circuit/mock/test/mock_circuit_light.py
	lineno: 	104
	code_context: 	['        log.get_source_info(),\n']
	index: 0

get excitatory cell density for layer 6
with number of excitatory mtypes: 4

test develop mock circuit Logger                                                
DEBUGINFO@<2020-02-18 13:36:54>                                                 
Traceback:
	filename: 	/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/models/bluebrain/circuit/mock/test/mock_circuit_light.py
	lineno: 	104
	code_context: 	['        log.get_source_info(),\n']
	index: 0

get excitatory cell density for layer 6
with number of excitatory mtypes: 4

test develop mock circuit Logger                    

43it [00:00, 422.92it/s]

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/models/bluebrain/circuit/mock/builder.py Logger
DEBUGINFO@<2020-02-18 13:36:54>                                                 
Build the (mock) circuit connectome.



82304it [03:11, 429.53it/s]
100%|██████████| 82304/82304 [01:20<00:00, 1028.39it/s]


# Adapter Functionality

In this section we develop our adapter's functionality

In [5]:
from neuro_dmt.models.bluebrain.circuit.adapter\
    import BlueBrainCircuitAdapter
adapter = BlueBrainCircuitAdapter()
cell_types_m = adapter.get_cell_types(mock_circuit_model, ("mtype",))
print(cell_types_m.head())
cell_types_me = adapter.get_cell_types(mock_circuit_model, ("mtype", "etype",) )
print(cell_types_me.head())

     mtype
0   L1_DAC
1  L1_SLAC
2   L23_MC
3  L23_ChC
4  L23_SBC
     mtype etype
0   L1_DAC  bNAC
1   L1_DAC   NAC
2  L1_SLAC  bNAC
3  L1_SLAC   NAC
4   L23_MC  bNAC


In [11]:
pathways = adapter.get_pathways(mock_circuit_model, ("mtype", "etype"))
print(pathways.shape)
assert pathways.shape[0] == cell_types_me.shape[0] ** 2
pathways.head()

(1444, 4)


pre_synaptic       post_synaptic      
         mtype etype         mtype etype
0       L1_DAC  bNAC        L1_DAC  bNAC
1       L1_DAC  bNAC        L1_DAC   NAC
2       L1_DAC  bNAC       L1_SLAC  bNAC
3       L1_DAC  bNAC       L1_SLAC   NAC
4       L1_DAC  bNAC        L23_MC  bNAC

In [12]:
pathways.iloc[1]

pre_synaptic   mtype    L1_DAC
               etype      bNAC
post_synaptic  mtype    L1_DAC
               etype       NAC
Name: 1, dtype: object

In [6]:
adapter.get_cells(mock_circuit_model,
                  **pathways.iloc[1].post_synaptic)\
        .etype\
        .unique()

NameError: name 'pathways' is not defined

# Number of Afferent Connections

We develop an analysis of the number of afferent connections

In [14]:
from neuro_dmt.analysis.reporting import CheetahReporter
reporter_analyses =\
    CheetahReporter(
        path_output_folder=os.path.join(
            suite_connectome_analyses.path_reports,
            "analyses"))

We can also run our analyses over a chosen set of cell-types.

In [7]:
post_synaptic_cell =\
    suite_connectome_analyses\
        .random_cell(
            mock_circuit_model,
            adapter,
            {"mtype": "L23_MC"})
post_synaptic_cell

region                  S1Tr
layer                      3
nucleus          not-defined
mtype                 L23_MC
etype                    NAC
morph_class      not-defined
synapse_class            INH
x                    9.28287
y                    324.033
z                    99.8534
gid                    42046
Name: 42046, dtype: object

In [8]:
m =\
    suite_connectome_analyses\
        .number_connections_afferent(
            mock_circuit_model, adapter,
            {"mtype": "L23_MC"})
m.head()

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 13:36:54>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            



(pre_synaptic_cell_type, mtype)  soma_distance
L1_DAC                           150.0            1.0
L23_ChC                          50.0             2.0
                                 150.0            1.0
                                 250.0            3.0
                                 450.0            1.0
Name: 62110, dtype: float64

In [ ]:
ms =\
    suite_connectome_analyses\
      .summary_number_connections_afferent(
          mock_circuit_model, adapter,
          {"mtype": "L23_MC"},
          sample_size=100)
print(ms.shape)
ms.head()

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 13:36:54>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            



In [59]:
suite_connectome_analyses =\
    ConnectomeAnalysesSuite(
        sample_size=20,
        cell_mtypes=["L5_TPC:A", "L5_TPC:B", "L5_UPC"])
ms = pd.concat(
    [suite_connectome_analyses\
        .number_afferent_connections(
            mock_circuit_model, adapter,
            {"mtype": "L23_MC"})
        .rename(i)
        for i in range(100)],
    axis=1)
    #keys=["m{}".format(i) for i in range(100)])

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': '

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': '

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': '

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': '

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': '

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': '

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-18 09:51:21>                                                 

            Number afferent connections for post-synaptic cell type: {'mtype': '

In [60]:
ms.head()

0    1    2    3    4    5   \
(pre_synaptic_cell_type, mtype) soma_distance                                 
L1_DAC                          50.0           NaN  NaN  NaN  NaN  NaN  NaN   
                                150.0          1.0  1.0  NaN  NaN  1.0  5.0   
                                250.0          2.0  NaN  1.0  NaN  NaN  1.0   
                                350.0          NaN  NaN  NaN  NaN  NaN  NaN   
                                450.0          NaN  NaN  NaN  1.0  NaN  NaN   

                                                6    7    8    9   ...   90  \
(pre_synaptic_cell_type, mtype) soma_distance                      ...        
L1_DAC                          50.0           NaN  NaN  NaN  NaN  ...  NaN   
                                150.0          NaN  1.0  2.0  1.0  ...  1.0   
                                250.0          NaN  NaN  NaN  NaN  ...  NaN   
                                350.0          NaN  NaN  NaN  NaN  ...  NaN   
                                450.0          1.0  NaN  NaN  NaN  ...  NaN   

                                                91   92   93   94   95   96  \
(pre_synaptic_cell_type, mtype) soma_distance                                 
L1_DAC                          50.0           NaN  NaN  NaN  NaN  NaN  1.0   
                                150.0          3.0  2.0  1.0  2.0  NaN  2.0   
                                250.0          NaN  2.0  1.0  1.0  NaN  NaN   
                                350.0          NaN  NaN  NaN  NaN  2.0  NaN   
                                450.0          NaN  NaN  NaN  NaN  NaN  NaN   

                                                97  98   99  
(pre_synaptic_cell_type, mtype) soma_distance                
L1_DAC                          50.0           NaN NaN  NaN  
                                150.0          NaN NaN  2.0  
                                250.0          NaN NaN  NaN  
                                350.0          1.0 NaN  NaN  
                                450.0          NaN NaN  NaN  

[5 rows x 100 columns]

In [47]:
ams = ms.agg(["count", "sum", "mean", "std", "min", "median", "max", "mad"], axis=1)
ams.index = ms.index

In [48]:
ams.head()

count   sum      mean  \
(pre_synaptic_cell_type, mtype) soma_distance                          
L1_DAC                          50.0             4.0   4.0  1.000000   
                                150.0           37.0  54.0  1.459459   
                                250.0           32.0  42.0  1.312500   
                                350.0           20.0  29.0  1.450000   
                                450.0           14.0  19.0  1.357143   

                                                    std  min  median  max  \
(pre_synaptic_cell_type, mtype) soma_distance                               
L1_DAC                          50.0           0.000000  1.0     1.0  1.0   
                                150.0          0.836481  1.0     1.0  5.0   
                                250.0          0.592289  1.0     1.0  3.0   
                                350.0          0.759155  1.0     1.0  4.0   
                                450.0          0.744946  1.0     1.0  3.0   

                                                    mad  
(pre_synaptic_cell_type, mtype) soma_distance            
L1_DAC                          50.0           0.000000  
                                150.0          0.620891  
                                250.0          0.468750  
                                350.0          0.585000  
                                450.0          0.561224

In [21]:
ms.groupby([("pre_synaptic_cell_type", "mtype"), ("soma_distance")])\
  .agg(["size", "sum", "mean", "std", "min", "median", "max"])

size    sum      mean  \
(pre_synaptic_cell_type, mtype) soma_distance                          
L1_DAC                          50.0              6    6.0  1.000000   
                                150.0            24   34.0  1.416667   
                                250.0            25   42.0  1.680000   
                                350.0            14   19.0  1.357143   
                                450.0            16   22.0  1.375000   
...                                             ...    ...       ...   
L6_UPC                          1350.0           97  330.0  3.402062   
                                1450.0           78  238.0  3.051282   
                                1550.0           63  199.0  3.158730   
                                1650.0           44  118.0  2.681818   
                                1750.0            1    1.0  1.000000   

                                                    std  min  median   max  
(pre_synaptic_cell_type, mtype) soma_distance                               
L1_DAC                          50.0           0.000000  1.0     1.0   1.0  
                                150.0          0.880547  1.0     1.0   5.0  
                                250.0          0.988264  1.0     1.0   5.0  
                                350.0          0.633324  1.0     1.0   3.0  
                                450.0          0.619139  1.0     1.0   3.0  
...                                                 ...  ...     ...   ...  
L6_UPC                          1350.0         2.024124  1.0     3.0  10.0  
                                1450.0         1.650639  1.0     3.0   8.0  
                                1550.0         1.927780  1.0     3.0  10.0  
                                1650.0         2.164972  1.0     2.0  10.0  
                                1750.0              NaN  1.0     1.0   1.0  

[159 rows x 7 columns]

In [36]:
from tqdm import tqdm
connectome = mock_circuit_model.connectome
def iter_connections(pre=None, post=None):
    if pre is None:
        if post is not None:
            for post_gid in post:
                afferent_connections = connectome.afferent_connections.loc[post_gid]
                for _, cnxn in afferent_connections.iterrows():
                    yield (cnxn.pre_synaptic_cell_gid, post_gid, cnxn.strength)
        else:
            for _, cnxn in connectome.connections.iterrows():
                yield (cnxn.pre_synaptic_cell_gid, cnxn.post_synaptic_cell_gid, cnxn.strength)
    elif post is None:
        for pre_gid in pre:
            efferent_connections = connectome.efferent_connections.loc[pre_gid]
            for _, cnxn in efferent_connections.iterrows():
                yield (pre_gid, cnxn.post_synaptic_cell_gid, cnxn.strength)
            
    else:
        for pre_gid in pre:
            efferent_synapse_count =\
                connectome.synapse_count\
                          .loc[pre_gid]\
                          .reindex(post)\
                          .dropna()
            for post_gid, strength in efferent_synapse_count.iteritems():
                yield (pre_gid, post_gid, strength)

In [41]:
iterator =\
    iter_connections(post=[post_synaptic_cell.gid])
                    
df =\
    pd.DataFrame(
        [cnxn for cnxn in iterator],
        columns=["pre", "post", "strength"])
df

,pre,post,strength
0,525,62587,5
1,768,62587,4
2,832,62587,10
3,1260,62587,2
4,1413,62587,4
...,...,...,...
199,80484,62587,1
200,81335,62587,3
201,81816,62587,6
202,81857,62587,6


In [40]:
pd.DataFrame(
    [cnxn for cnxn in iter_connections(pre=[post_synaptic_cell.gid])],
    columns=["pre", "post", "strength"])

,pre,post,strength
0,62587,289,4
1,62587,501,4
2,62587,656,5
3,62587,1162,6
4,62587,1727,4
...,...,...,...
181,62587,80333,4
182,62587,80601,3
183,62587,80854,3
184,62587,80981,2


In [17]:
adapter.get_afferent_connections(mock_circuit_model, post_synaptic_cell)

,pre_gid,strength
0,780,6
1,1539,6
2,2214,3
3,2348,5
4,2535,3
...,...,...
182,79365,2
183,80498,6
184,81130,7
185,81325,2


In [21]:
m = suite_connectome_analyses\
     .strength_afferent_connections(
        mock_circuit_model, adapter,
        {"mtype": "L23_MC"})

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
DEBUGINFO@<2020-02-17 22:55:05>                                                 

            Strength afferent connections for post-synaptic cell type: {'mtype': 'L23_MC'}
            

/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/neuro_dmt/library/users/visood/sscx_dissemination/analyses/connectome/__init__.py Logger
INFO@<2020-02-17 22:55:05>                                                      
strength_afferent_connections call count : 2



In [22]:
m.shape

(69,)

## SSCx variation

In [ ]:
report_connectome_analyses_bio_m =\
    suite_connectome_analyses\
        .analysis_number_afferent_connections(
            circuit_bio_m,
            adapter)


Because 60 post-synaptic mtypes can take too long, we can set the number of 
mtypes to process in one run:

In [ ]:
def get_layer_mtypes(circuit, query_layer):
    mtypes_all =\
        pd.Series(adapter.get_mtypes(circuit), name="mtype")
    return\
        list(mtypes_all[
                [query_layer in mtype for mtype in mtypes_all]
        ].values)
def analyze_number_afferent_connections(
        circuit,
        adapter,
        selected_mtypes,
        reporter,
        **kwargs):
    """..."""
    suite_connectome_analyses =\
        ConnectomeAnalysesSuite(
            sample_size=20,
            cell_mtypes=selected_mtypes)
    report=\
        suite_connectome_analyses\
            .analysis_number_afferent_connections(
                circuit, adapter, **kwargs)
    return reporter.post(report)

In [ ]:
analyze_number_afferent_connections(
    mock_circuit_model,
    adapter,
    ["L5_TPC:A", "L5_TPC:B",  "L5_UPC"],
    reporter_analyses,
    sample_size=200)

In [ ]:
suite_connectome_analyses =\
    ConnectomeAnalysesSuite(
        sample_size=20,
        cell_mtypes=["L23_MC"])
measurement_example =\
    suite_connectome_analyses\
        .analysis_number_afferent_connections\
        .get_measurement(
            mock_circuit_model,
            adapter)

In [ ]:
measurement_example["data"]